# make XOR gate

In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
print('torch ver: ',torch.__version__)
print("pyli ver: ",pl.__version__   )

torch ver:  2.2.0+cu121
pyli ver:  2.1.4


## 데이터 준비

In [3]:
# 우선 진리표르 만든다 (논리연산)

XOR_input = [torch.Tensor([0, 0]),
             torch.Tensor([0, 1]),
             torch.Tensor([1, 0]),
             torch.Tensor([1, 1])]

XOR_target = [torch.Tensor([0]),
              torch.Tensor([1]),
              torch.Tensor([1]),
              torch.Tensor([0])]


In [4]:
xor_data = list(zip(XOR_input, XOR_target))
trian_loader = DataLoader(xor_data, batch_size=1)

In [5]:
print(xor_data)

[(tensor([0., 0.]), tensor([0.])), (tensor([0., 1.]), tensor([1.])), (tensor([1., 0.]), tensor([1.])), (tensor([1., 1.]), tensor([0.]))]


## 모델선택

In [6]:
# 모델을 고르자, XOR 이니까 MLP정도가 적당할것.
from typing import Any


class XORModel(pl.LightningModule):
    def __init__ (self):
        super(XORModel,self).__init__()
        self.input_layer = nn.Linear(2,4) # 첫 잠재층. 2개를 입력받아 4개 출력을 반환한다.
        self.output_layer = nn.Linear(4,1) # 반환된 4개의 출력물을 1개로 취합한다. 이는 출력노드가 된다.
        self.sigmoid = nn.Sigmoid() # 활성함수는 sigmoid
        self.loss = nn.BCELoss() # 로스는 MSE

# 모델 입력 전달
    def forward(self, x_input):
        x = self.input_layer(x_input)
        x = self.sigmoid(x)
        x = self.output_layer(x)
        x = self.sigmoid(x)
        return x
# 옵티마이저 설정
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.01)
        return optimizer
# 학습파라미터 설정
    def training_step(self, batch, batch_idx):
        xor_input, xor_target = batch
        outputs = self(xor_input)
        loss = self.loss(outputs, xor_target)
        return loss

    
    

## 모델학습하기

In [7]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
checkpoint_callback = ModelCheckpoint()
model = XORModel()

trainer = pl.Trainer(accelerator="auto", devices="auto", max_epochs=100, callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/workspace/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
trainer.fit(model, train_dataloaders=trian_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | BCELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)
/workspace/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasi

Epoch 99: 100%|██████████| 4/4 [00:00<00:00, 283.83it/s, v_num=6]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 4/4 [00:00<00:00, 229.56it/s, v_num=6]


### 학습된 xor 모델
주석을 달면 오류가 남으로 마크다운으로 설명. 
학습된 모델과 그 하위 폴더에있는 파일을 보여줌, 한번더 학습하면 모델이 쌓인다.

In [56]:
ls lightning_logs/


version_0/ version_1/


In [57]:
ls lightning_logs/*/

lightning_logs/version_0/:
checkpoints/  hparams.yaml

lightning_logs/version_1/:
checkpoints/  hparams.yaml


## 모델 불러오기
버전이 둘이라면, 어떤 버전의 모델을 불러올지 결정해야한다. 그때 쓰는게 checkpoint_callback.best_model_path 이다

In [58]:
print(checkpoint_callback.best_model_path)
# 결과를 보면 최신 모델의 경로를 얻을수있다.

/Users/ryujekyeong/Documents/pytorchLi/pyli/lightning_logs/version_1/checkpoints/epoch=99-step=400.ckpt


In [64]:
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)

# 매서드를 클래시의 인스턴스에 대해 호출하면 오류가 생김, 따라서 클래스 자체를 호출해야한다.
#train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path) --> X


TypeError: The classmethod `XORModel.load_from_checkpoint` cannot be called on an instance. Please call it on the class type and make sure the return value is used.

In [65]:
test = torch.utils.data.DataLoader(xor_data, batch_size=1)
for val in XOR_input:
    prediction = train_model(val)
    print(val[0], val[1], prediction)

RuntimeError: Placeholder storage has not been allocated on MPS device!